# T3

# policy gradient

In [58]:
import warnings
warnings.filterwarnings("ignore")

In [59]:
import torch 
import torch.nn as nn
import numpy as np
import gym
from torch.optim import AdamW

**1. Parametrización de política**

In [60]:
class Policy(nn.Module):
    
    def __init__(self, dim_states, dim_actions, continuous_control):
        super(Policy, self).__init__()
        # MLP, fully connected layers, ReLU activations, linear ouput activation
        # dim_states -> 64 -> 64 -> dim_actions

        self.layers = nn.Sequential(
            nn.Linear(dim_states, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, dim_actions)
        )
        
        if continuous_control:
            # trainable parameter
            self.log_std = nn.Parameter(torch.zeros(1, dim_actions))


    def forward(self, input):

        # tensor format
        if isinstance(input, torch.Tensor):
            input=input
            
        else:
            input = torch.from_numpy(input).unsqueeze(dim=0).float()
            
        value = self.layers(input)
        
        return value


In [61]:
env = gym.make('Pendulum-v1')
dim_states = env.observation_space.shape[0]
continuous_control = isinstance(env.action_space, gym.spaces.Box)
dim_actions = env.action_space.shape[0] if continuous_control else env.action_space.n
print(dim_states, dim_actions,continuous_control)

3 1 True


In [62]:
env.action_space.sample()

array([-1.5033903], dtype=float32)

In [63]:
RN_policy= Policy(dim_states, dim_actions,continuous_control)
RN_policy

Policy(
  (layers): Sequential(
    (0): Linear(in_features=3, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [64]:
RN_policy.log_std

Parameter containing:
tensor([[0.]], requires_grad=True)

In [65]:
s_t=env.reset()
s_t

array([0.5429128 , 0.8397891 , 0.39806354], dtype=float32)

In [66]:
action=RN_policy(s_t)
action

tensor([[-0.0039]], grad_fn=<AddmmBackward>)

In [67]:
class PolicyGradients:
    
    def __init__(self, dim_states, dim_actions, lr, gamma, 
                 continuous_control=False, reward_to_go=False, use_baseline=False):
        
        self._learning_rate = lr
        self._gamma = gamma
        
        self._dim_states = dim_states
        self._dim_actions = dim_actions

        self._continuous_control = continuous_control
        self._use_reward_to_go = reward_to_go
        self._use_baseline = use_baseline

        self._policy = Policy(self._dim_states, self._dim_actions, self._continuous_control)
        # Adam optimizer
        self._optimizer = AdamW(self._policy.parameters(), lr=self._learning_rate)

        self._select_action = self._select_action_continuous if self._continuous_control else self._select_action_discrete
        self._compute_loss = self._compute_loss_continuous if self._continuous_control else self._compute_loss_discrete


    def select_action(self, observation):
        return self._select_action(observation)
        

    def _select_action_discrete(self, observation):
        # sample from categorical distribution
        RN_policy=self._policy 
        logits=RN_policy(observation)

        # Probabilidad de cada acción
        probs = torch.softmax(logits, dim=-1)

        # Distribución de probabilidad categorica
        dist = torch.distributions.Categorical(probs)

        # Sample de acción
        action = dist.sample()#.item()
     
        return action


    def _select_action_continuous(self, observation):
        # sample from normal distribution
        # use the log std trainable parameter

        # RN
        RN_policy=self._policy

        # Parametro log std de la RN
        log_std=RN_policy.log_std
        std = torch.exp(log_std)

        # Politica dada la observación (Representa el promedio de la distribución normal que muestrea acciones)
        policy=RN_policy(observation)
        
        # Distribución normal de parametros mean y std, esta se utiliza para muestrear acciones de modo de tal de explorar el espacio de acciones
        dist = torch.distributions.Normal(policy, std)

        # sample de acción
        action = dist.sample()
        
        # Asegurarse de que las acciones están dentro del rango [-1, 1]
        #action = torch.tanh(action)
        
        return action
            

    def update(self, observation_batch, action_batch, advantage_batch):
        # update the policy here
        # you should use self._compute_loss 

        pass
    

    def _compute_loss_discrete(self, observation_batch, action_batch, advantage_batch):
        # use negative logprobs * advantages
        pass


    def _compute_loss_continuous(self, observation_batch, action_batch, advantage_batch):
        # use negative logprobs * advantages
        pass

    
    def estimate_returns(self, rollouts_rew):
        estimated_returns = []
        for rollout_rew in rollouts_rew:
                
            if self._use_reward_to_go:
                # only for part 2
                estimated_return = None
            else:
                estimated_return = None
            
            estimated_returns = np.concatenate([estimated_returns, estimated_return])

        if self._use_baseline:
            # only for part 2
            average_return_baseline = None
            # Use the baseline:
            #estimated_returns -= average_return_baseline

        return np.array(estimated_returns, dtype=np.float32)


    # It may be useful to discount the rewards using an auxiliary function [optional]
    def _discount_rewards(self, rewards):
        pass


**2. Muestreo de trayectorias**

In [68]:
import gym
import time
import datetime
import csv

import numpy as np

import matplotlib.pyplot as plt

#from policy_gradients import PolicyGradients


def perform_single_rollout(env, agent, episode_nb, render=False):

    # Modify this function to return a tuple of numpy arrays containing (observations, actions, rewards).
    # (np.array(obs), np.array(acs), np.array(rws))
    # np.array(obs) -> shape: (time_steps, nb_obs)
    # np.array(acs) -> shape: (time_steps, nb_acs) if actions are continuous, (time_steps,) if actions are discrete
    # np.array(rws) -> shape: (time_steps,)

    obs_list = []
    action_list = []
    reward_list = []

    ob_t = env.reset()
    
    done = False
    episode_reward = 0
    nb_steps = 0

    while not done:
        
        if render:
            env.render()
            time.sleep(1. / 60)

        #action = agent.select_action(ob_t)
        
        action = agent.select_action(ob_t)
    
        #print(action)

        try:    
            ob_t1, reward, done, _ = env.step(action)

        except:
            ob_t1, reward, done, _ = env.step(action.item())


        obs_list.append(ob_t1)
        action_list.append(action)
        reward_list.append(reward)

        ob_t = np.squeeze(ob_t1) # <-- may not be needed depending on gym version

        episode_reward += reward
        
        nb_steps += 1

        if done:
            print(f"Largo del episodio {nb_steps}")
            obs_array = np.array(obs_list)
            action_array = np.array(action_list)
            reward_array = np.array(reward_list)

            return obs_array, action_array, reward_array
    #return None

def sample_rollouts(env, agent, training_iter, min_batch_steps):

    sampled_rollouts = []
    total_nb_steps = 0
    episode_nb = 0
    
    while total_nb_steps < min_batch_steps:

        episode_nb += 1
        #render = training_iter%10 == 0 and len(sampled_rollouts) == 0 # Change training_iter%10 to any number you want
        render=False
        # Use perform_single_rollout to get data 
        # Uncomment once perform_single_rollout works.
        # Return sampled_rollouts
       
        sample_rollout = perform_single_rollout(env, agent, episode_nb, render=render)
        total_nb_steps += len(sample_rollout[0])

        sampled_rollouts.append(sample_rollout)
        
    return sampled_rollouts




In [69]:
env = gym.make('Pendulum-v1')

dim_states = env.observation_space.shape[0]

continuous_control = isinstance(env.action_space, gym.spaces.Box)
dim_actions = env.action_space.shape[0] if continuous_control else env.action_space.n

policy_gradients_agent = PolicyGradients(dim_states=dim_states, 
                                             dim_actions=dim_actions, 
                                             lr=0.005,
                                             gamma=0.99,
                                             continuous_control=continuous_control,
                                             reward_to_go=False,
                                             use_baseline=False)



In [70]:
# Rollout
x1=perform_single_rollout(env, policy_gradients_agent, 1000, render=False)
print(x1[0].shape)
print(x1[1].shape)
print(x1[2].shape)

Largo del episodio 200
(200, 3)
(200,)
(200,)


**El número de filas de las observaciones es igual con el largo del episodio, por lo que se concluye el correcto funcionamiento de la función.**

In [71]:
# Sample rollouts
x2=sample_rollouts(env, policy_gradients_agent, 1000, 5000)

Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200


In [72]:
sampled_obs = np.concatenate([x2[i][0] for i in range(len(x2))])
sampled_action = np.concatenate([x2[i][1] for i in range(len(x2))])
sampled_reward = np.concatenate([x2[i][2] for i in range(len(x2))])
print(sampled_obs.shape)
print(sampled_action.shape)
print(sampled_reward.shape)

(5000, 3)
(5000,)
(5000,)


**El largo del registro de sample rollout es al menos el número de sample mini batch, se concluye que la función funciona.**

In [204]:
env = gym.make('CartPole-v1')

dim_states = env.observation_space.shape[0]

continuous_control = isinstance(env.action_space, gym.spaces.Box)
dim_actions = env.action_space.shape[0] if continuous_control else env.action_space.n

policy_gradients_agent = PolicyGradients(dim_states=dim_states, 
                                             dim_actions=dim_actions, 
                                             lr=0.005,
                                             gamma=0.99,
                                             continuous_control=continuous_control,
                                             reward_to_go=False,
                                             use_baseline=False)


In [205]:
# Rollout
x1=perform_single_rollout(env, policy_gradients_agent, 1000, render=False)
print(x1[0].shape)
print(x1[1].shape)
print(x1[2].shape)

Largo del episodio 10
(10, 4)
(10,)
(10,)


**El número de filas de las observaciones es igual con el largo del episodio, por lo que se concluye el correcto funcionamiento de la función.**

In [206]:
# Sample rollouts
x2=sample_rollouts(env, policy_gradients_agent, 1000, 5000)

Largo del episodio 9
Largo del episodio 54
Largo del episodio 10
Largo del episodio 28
Largo del episodio 17
Largo del episodio 19
Largo del episodio 24
Largo del episodio 20
Largo del episodio 20
Largo del episodio 11
Largo del episodio 27
Largo del episodio 29
Largo del episodio 52
Largo del episodio 18
Largo del episodio 19
Largo del episodio 18
Largo del episodio 9
Largo del episodio 14
Largo del episodio 17
Largo del episodio 29
Largo del episodio 23
Largo del episodio 33
Largo del episodio 13
Largo del episodio 47
Largo del episodio 19
Largo del episodio 53
Largo del episodio 12
Largo del episodio 17
Largo del episodio 17
Largo del episodio 21
Largo del episodio 17
Largo del episodio 14
Largo del episodio 9
Largo del episodio 21
Largo del episodio 28
Largo del episodio 21
Largo del episodio 21
Largo del episodio 25
Largo del episodio 28
Largo del episodio 15
Largo del episodio 39
Largo del episodio 21
Largo del episodio 19
Largo del episodio 11
Largo del episodio 23
Largo del epi

In [207]:
sampled_obs = np.concatenate([x2[i][0] for i in range(len(x2))])
sampled_action = np.concatenate([x2[i][1] for i in range(len(x2))])
sampled_reward = np.concatenate([x2[i][2] for i in range(len(x2))])
print(sampled_obs.shape)
print(sampled_action.shape)
print(sampled_reward.shape)

(5014, 4)
(5014,)
(5014,)


In [211]:
x2[2]

(array([[-0.01524238,  0.22910602, -0.04746097, -0.3423483 ],
        [-0.01066026,  0.42486995, -0.05430794, -0.64961183],
        [-0.00216286,  0.62070465, -0.06730018, -0.9588895 ],
        [ 0.01025123,  0.4265489 , -0.08647797, -0.68808645],
        [ 0.01878221,  0.6227579 , -0.10023969, -1.006693  ],
        [ 0.03123737,  0.81906503, -0.12037355, -1.3290967 ],
        [ 0.04761866,  1.0154825 , -0.14695549, -1.6568954 ],
        [ 0.06792831,  0.8223487 , -0.1800934 , -1.4133668 ],
        [ 0.08437529,  1.0191867 , -0.20836073, -1.7565104 ],
        [ 0.10475902,  1.2159724 , -0.24349093, -2.1061203 ]],
       dtype=float32),
 array([1, 1, 1, 0, 1, 1, 1, 0, 1, 1], dtype=int64),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]))

In [209]:
sampled_reward

array([1., 1., 1., ..., 1., 1., 1.])

**El largo del registro de sample rollout es al menos el número de sample mini batch, se concluye que la función funciona.**

**3. Estimación de retornos**

In [77]:
def estimate_returns( rollouts_rew):
        estimated_returns = []
        for rollout_rew in rollouts_rew:

            # Largo del episodio (largo del reward)
            n_steps = len(rollout_rew[2])
            estimated_return = np.zeros(n_steps)

            if _use_reward_to_go:
            
                estimated_return = None
            else:
                estimated_return = np.zeros(n_steps)

                vec_gammas=np.array([_gamma**j for j in range(n_steps)])

                sum_descount=np.sum(vec_gammas*rollout_rew[2])

                for t in range(n_steps):
                    
                    estimated_return[t] = sum_descount
                    
             
            estimated_returns = np.concatenate([estimated_returns, estimated_return])

        if _use_baseline:
            pass
            #average_return_baseline = np.mean(estimated_returns)
            #estimated_returns -= average_return_baseline

        return np.array(estimated_returns, dtype=np.float32)


In [ ]:
print(estimate_returns(x2))

In [78]:
_gamma=0.99
_use_reward_to_go=False
_use_baseline=False

In [79]:
env = gym.make('CartPole-v1')

dim_states = env.observation_space.shape[0]

continuous_control = isinstance(env.action_space, gym.spaces.Box)
dim_actions = env.action_space.shape[0] if continuous_control else env.action_space.n

policy_gradients_agent = PolicyGradients(dim_states=dim_states, 
                                             dim_actions=dim_actions, 
                                             lr=0.005,
                                             gamma=0.99,
                                             continuous_control=continuous_control,
                                             reward_to_go=False,
                                             use_baseline=False)

# Sample rollouts (2 episodios): Ejecutar hasta que se generen solo 2 episodios!!
x2=sample_rollouts(env, policy_gradients_agent, 1000, 22)
print("")
print("Vector de retorno")
print(estimate_returns(x2))
print("")
retorno=0
for t,reward in enumerate(x2[0][2]):
    retorno=retorno+(_gamma**t)*reward
print("Retorno Ep 1")
print(retorno)
print("")
retorno=0
for t,reward in enumerate(x2[1][2]):
    retorno=retorno+(_gamma**t)*reward
print("Retorno Ep 2")
print(retorno)
print("")

Largo del episodio 10
Largo del episodio 32

Vector de retorno
[ 9.561792  9.561792  9.561792  9.561792  9.561792  9.561792  9.561792
  9.561792  9.561792  9.561792 27.501966 27.501966 27.501966 27.501966
 27.501966 27.501966 27.501966 27.501966 27.501966 27.501966 27.501966
 27.501966 27.501966 27.501966 27.501966 27.501966 27.501966 27.501966
 27.501966 27.501966 27.501966 27.501966 27.501966 27.501966 27.501966
 27.501966 27.501966 27.501966 27.501966 27.501966 27.501966 27.501966]

Retorno Ep 1
9.561792499119552

Retorno Ep 2
27.501966404214624



In [80]:
env=gym.make('Pendulum-v1')

dim_states = env.observation_space.shape[0]

continuous_control = isinstance(env.action_space, gym.spaces.Box)
dim_actions = env.action_space.shape[0] if continuous_control else env.action_space.n

policy_gradients_agent = PolicyGradients(dim_states=dim_states, 
                                             dim_actions=dim_actions, 
                                             lr=0.005,
                                             gamma=0.99,
                                             continuous_control=continuous_control,
                                             reward_to_go=False,
                                             use_baseline=False)

# Sample rollouts (2 episodios): Ejecutar hasta que se generen solo 2 episodios!!
x2=sample_rollouts(env, policy_gradients_agent, 1000, 220)

index_episodio_1=x2[0][1].shape[0]

print("")
print("Muestra vector de retorno ep 1")
print(estimate_returns(x2)[index_episodio_1-1])
print("")
print("")
print("Muestra vector de retorno ep 2")
print(estimate_returns(x2)[index_episodio_1+1])
print("")

retorno=0
for t,reward in enumerate(x2[0][2]):
    retorno=retorno+(_gamma**t)*reward
print("Retorno Ep 1")
print(retorno)
print("")

retorno=0
for t,reward in enumerate(x2[1][2]):
    retorno=retorno+(_gamma**t)*reward
print("Retorno Ep 2")
print(retorno)
print("")

Largo del episodio 200
Largo del episodio 200

Muestra vector de retorno ep 1
-680.63947


Muestra vector de retorno ep 2
-767.5068

Retorno Ep 1
-680.6394373151065

Retorno Ep 2
-767.5067861033187



**Se observa que que para ambos ambientes y en cada episodio, los retornos coinciden los calculos obtenidos.**

**4. Policy gradients**

In [94]:
env = gym.make('CartPole-v1')
#env=gym.make('Pendulum-v1')
env.reset()
dim_states = env.observation_space.shape[0]

continuous_control = isinstance(env.action_space, gym.spaces.Box)
dim_actions = env.action_space.shape[0] if continuous_control else env.action_space.n

policy_gradients_agent = PolicyGradients(dim_states=dim_states, 
                                             dim_actions=dim_actions, 
                                             lr=0.005,
                                             gamma=0.99,
                                             continuous_control=continuous_control,
                                             reward_to_go=False,
                                             use_baseline=False)

In [82]:
# Sample rollouts (2 episodios): Ejecutar hasta que se generen solo 2 episodios!!
x2=sample_rollouts(env, policy_gradients_agent, 1000, 220)

Largo del episodio 24
Largo del episodio 12
Largo del episodio 15
Largo del episodio 59
Largo del episodio 23
Largo del episodio 47
Largo del episodio 31
Largo del episodio 24


In [95]:
sampled_obs = np.concatenate([x2[i][0] for i in range(len(x2))])
sampled_action = np.concatenate([x2[i][1] for i in range(len(x2))])
sampled_reward = np.concatenate([x2[i][2] for i in range(len(x2))])
print(sampled_obs.shape)
print(sampled_action.shape)
print(sampled_reward.shape)

(235, 4)
(235,)
(235,)


In [103]:
RN_policy= Policy(dim_states, dim_actions,continuous_control)
logits=RN_policy(sampled_obs)
logits

tensor([[[ 0.0582,  0.0596],
         [ 0.0718,  0.0593],
         [ 0.0842,  0.0629],
         [ 0.0717,  0.0582],
         [ 0.0602,  0.0585],
         [ 0.0501,  0.0643],
         [ 0.0602,  0.0579],
         [ 0.0482,  0.0625],
         [ 0.0523,  0.0660],
         [ 0.0518,  0.0665],
         [ 0.0457,  0.0645],
         [ 0.0402,  0.0639],
         [ 0.0337,  0.0596],
         [ 0.0236,  0.0492],
         [ 0.0330,  0.0590],
         [ 0.0224,  0.0484],
         [ 0.0135,  0.0358],
         [ 0.0228,  0.0459],
         [ 0.0288,  0.0542],
         [ 0.0237,  0.0430],
         [ 0.0280,  0.0506],
         [ 0.0250,  0.0398],
         [ 0.0122,  0.0248],
         [ 0.0240,  0.0343],
         [ 0.0723,  0.0564],
         [ 0.0862,  0.0574],
         [ 0.0987,  0.0585],
         [ 0.0875,  0.0556],
         [ 0.0982,  0.0554],
         [ 0.1015,  0.0456],
         [ 0.0976,  0.0509],
         [ 0.0903,  0.0501],
         [ 0.0953,  0.0464],
         [ 0.0962,  0.0355],
         [ 0.0

In [120]:
# Distribución de probabilidad categorica
dist = torch.distributions.Categorical(logits=logits)
log_probs=dist.log_prob(torch.tensor(sampled_action)).squeeze(0)
log_probs

tensor([-0.6939, -0.6869, -0.6826, -0.6999, -0.6940, -0.6861, -0.6920, -0.6860,
        -0.6863, -0.6859, -0.6838, -0.6813, -0.6803, -0.6804, -0.7063, -0.6802,
        -0.6821, -0.7048, -0.7059, -0.6836, -0.7045, -0.6858, -0.6869, -0.6983,
        -0.6852, -0.6789, -0.6733, -0.7092, -0.6720, -0.6656, -0.7168, -0.7134,
        -0.6690, -0.6632, -0.7199, -0.6615, -0.6897, -0.6947, -0.6852, -0.6921,
        -0.6917, -0.6909, -0.6935, -0.6935, -0.6942, -0.6939, -0.6934, -0.6936,
        -0.6913, -0.6878, -0.6969, -0.6897, -0.6890, -0.6871, -0.6977, -0.6866,
        -0.6862, -0.7005, -0.6857, -0.6860, -0.7012, -0.7018, -0.6852, -0.6857,
        -0.7006, -0.7024, -0.7007, -0.6926, -0.6862, -0.6924, -0.6866, -0.6859,
        -0.6996, -0.6923, -0.6872, -0.6875, -0.6990, -0.6883, -0.6986, -0.6893,
        -0.6981, -0.6905, -0.6914, -0.6943, -0.6971, -0.6916, -0.6827, -0.6795,
        -0.7035, -0.6971, -0.6923, -0.6891, -0.6822, -0.6754, -0.6709, -0.7133,
        -0.6696, -0.6620, -0.6506, -0.72

In [121]:
retorno_descontado=torch.tensor(estimate_returns(x2))
retorno_descontado

tensor([21.4322, 21.4322, 21.4322, 21.4322, 21.4322, 21.4322, 21.4322, 21.4322,
        21.4322, 21.4322, 21.4322, 21.4322, 21.4322, 21.4322, 21.4322, 21.4322,
        21.4322, 21.4322, 21.4322, 21.4322, 21.4322, 21.4322, 21.4322, 21.4322,
        11.3615, 11.3615, 11.3615, 11.3615, 11.3615, 11.3615, 11.3615, 11.3615,
        11.3615, 11.3615, 11.3615, 11.3615, 13.9942, 13.9942, 13.9942, 13.9942,
        13.9942, 13.9942, 13.9942, 13.9942, 13.9942, 13.9942, 13.9942, 13.9942,
        13.9942, 13.9942, 13.9942, 44.7317, 44.7317, 44.7317, 44.7317, 44.7317,
        44.7317, 44.7317, 44.7317, 44.7317, 44.7317, 44.7317, 44.7317, 44.7317,
        44.7317, 44.7317, 44.7317, 44.7317, 44.7317, 44.7317, 44.7317, 44.7317,
        44.7317, 44.7317, 44.7317, 44.7317, 44.7317, 44.7317, 44.7317, 44.7317,
        44.7317, 44.7317, 44.7317, 44.7317, 44.7317, 44.7317, 44.7317, 44.7317,
        44.7317, 44.7317, 44.7317, 44.7317, 44.7317, 44.7317, 44.7317, 44.7317,
        44.7317, 44.7317, 44.7317, 44.73

In [129]:
loss=torch.mean(log_probs*retorno_descontado)#.item()
loss

tensor(-20.8264, grad_fn=<MeanBackward0>)

In [130]:
optimizer=AdamW(RN_policy.parameters(), lr=0.001)

In [131]:
RN_policy.zero_grad()
loss.backward()
optimizer.step()

----------

In [132]:
env = gym.make('Pendulum-v1')
#env=gym.make('Pendulum-v1')
env.reset()
dim_states = env.observation_space.shape[0]

continuous_control = isinstance(env.action_space, gym.spaces.Box)
dim_actions = env.action_space.shape[0] if continuous_control else env.action_space.n

policy_gradients_agent = PolicyGradients(dim_states=dim_states, 
                                             dim_actions=dim_actions, 
                                             lr=0.005,
                                             gamma=0.99,
                                             continuous_control=continuous_control,
                                             reward_to_go=False,
                                             use_baseline=False)

In [133]:
# Sample rollouts (2 episodios): Ejecutar hasta que se generen solo 2 episodios!!
x2=sample_rollouts(env, policy_gradients_agent, 1000, 220)

Largo del episodio 200
Largo del episodio 200


In [134]:
sampled_obs = np.concatenate([x2[i][0] for i in range(len(x2))])
sampled_action = np.concatenate([x2[i][1] for i in range(len(x2))])
sampled_reward = np.concatenate([x2[i][2] for i in range(len(x2))])
print(sampled_obs.shape)
print(sampled_action.shape)
print(sampled_reward.shape)

(400, 3)
(400,)
(400,)


In [176]:
RN_policy= Policy(dim_states, dim_actions,continuous_control)
means=RN_policy(sampled_obs).squeeze(0).squeeze(1)
means.shape

torch.Size([400])

In [177]:
log_std=RN_policy.log_std
std = torch.exp(log_std)
std

tensor([[1.]], grad_fn=<ExpBackward>)

In [178]:
# Distribución normal de parametros mean y std, esta se utiliza para muestrear acciones de modo de tal de explorar el espacio de acciones
dist = torch.distributions.Normal(means, std)
dist


Normal(loc: torch.Size([1, 400]), scale: torch.Size([1, 400]))

In [185]:
log_probs=dist.log_prob(torch.tensor((sampled_action))).squeeze(0)
log_probs

tensor([-0.9404, -1.4960, -1.9028, -0.9539, -1.0602, -1.5006, -1.2621, -1.9320,
        -1.2645, -1.2595, -1.5775, -2.1709, -1.0486, -2.2181, -0.9416, -0.9203,
        -4.0093, -0.9334, -1.3452, -1.4623, -0.9794, -1.0039, -1.7908, -1.4091,
        -0.9412, -1.2732, -0.9807, -1.0128, -0.9714, -0.9297, -0.9668, -1.6230,
        -1.3276, -0.9492, -0.9805, -0.9537, -1.7452, -1.3581, -1.6791, -0.9561,
        -0.9889, -0.9464, -1.2974, -0.9749, -1.4319, -0.9923, -1.3504, -1.2266,
        -1.4641, -0.9384, -0.9571, -2.2671, -1.3252, -1.0737, -1.0106, -1.2892,
        -3.2430, -0.9278, -1.0241, -0.9314, -1.7817, -1.3425, -1.1508, -1.1592,
        -0.9660, -1.1511, -0.9284, -1.5336, -0.9408, -1.5021, -1.0373, -0.9775,
        -0.9190, -1.1696, -0.9346, -0.9312, -0.9662, -1.6375, -1.0595, -0.9330,
        -1.2317, -1.1437, -3.7218, -0.9479, -1.0424, -2.6386, -1.9198, -0.9449,
        -1.1156, -2.2420, -1.3952, -1.0752, -1.3470, -0.9466, -0.9247, -0.9357,
        -0.9861, -1.1646, -0.9190, -1.13

In [186]:
retorno_descontado=torch.tensor(estimate_returns(x2))
retorno_descontado

tensor([-628.5972, -628.5972, -628.5972, -628.5972, -628.5972, -628.5972,
        -628.5972, -628.5972, -628.5972, -628.5972, -628.5972, -628.5972,
        -628.5972, -628.5972, -628.5972, -628.5972, -628.5972, -628.5972,
        -628.5972, -628.5972, -628.5972, -628.5972, -628.5972, -628.5972,
        -628.5972, -628.5972, -628.5972, -628.5972, -628.5972, -628.5972,
        -628.5972, -628.5972, -628.5972, -628.5972, -628.5972, -628.5972,
        -628.5972, -628.5972, -628.5972, -628.5972, -628.5972, -628.5972,
        -628.5972, -628.5972, -628.5972, -628.5972, -628.5972, -628.5972,
        -628.5972, -628.5972, -628.5972, -628.5972, -628.5972, -628.5972,
        -628.5972, -628.5972, -628.5972, -628.5972, -628.5972, -628.5972,
        -628.5972, -628.5972, -628.5972, -628.5972, -628.5972, -628.5972,
        -628.5972, -628.5972, -628.5972, -628.5972, -628.5972, -628.5972,
        -628.5972, -628.5972, -628.5972, -628.5972, -628.5972, -628.5972,
        -628.5972, -628.5972, -628.597

In [187]:
loss=torch.mean(log_probs*retorno_descontado)#.item()
loss

tensor(772.3834, grad_fn=<MeanBackward0>)

In [188]:
optimizer=AdamW(RN_policy.parameters(), lr=0.001)

In [189]:
RN_policy.zero_grad()
loss.backward()
optimizer.step()

**5. Reducción de varianza**

In [27]:
def estimate_returns( rollouts_rew):
        estimated_returns = []
        for rollout_rew in rollouts_rew:

            # Largo del episodio (largo del reward)
            n_steps = len(rollout_rew[2])
            estimated_return = np.zeros(n_steps)

            if _use_reward_to_go:
                
                vec_gammas=np.array([_gamma**j for j in range(n_steps)])

                for t in range(n_steps):

                    sum_descount=np.sum(vec_gammas[t:]*rollout_rew[2][t:])
                    estimated_return[t] = sum_descount
    
            else:

                estimated_return = np.zeros(n_steps)

                vec_gammas=np.array([_gamma**j for j in range(n_steps)])

                sum_descount=np.sum(vec_gammas*rollout_rew[2])

                for t in range(n_steps):
                    
                    estimated_return[t] = sum_descount
                     
            estimated_returns = np.concatenate([estimated_returns, estimated_return])

        if _use_baseline:
            
            average_return_baseline = np.mean(estimated_returns)
            estimated_returns -= average_return_baseline

        return np.array(estimated_returns, dtype=np.float32)


In [28]:
env = gym.make('CartPole-v1')

dim_states = env.observation_space.shape[0]

continuous_control = isinstance(env.action_space, gym.spaces.Box)
dim_actions = env.action_space.shape[0] if continuous_control else env.action_space.n

policy_gradients_agent = PolicyGradients(dim_states=dim_states, 
                                             dim_actions=dim_actions, 
                                             lr=0.005,
                                             gamma=0.99,
                                             continuous_control=continuous_control,
                                             reward_to_go=False,
                                             use_baseline=False)

# Sample rollouts (2 episodios): Ejecutar hasta que se generen solo 2 episodios!!
x2=sample_rollouts(env, policy_gradients_agent, 1000, 22)

Largo del episodio 55


In [29]:
# Caso base
_gamma=0.99
_use_reward_to_go=False
_use_baseline=False
test=estimate_returns(x2)
test

array([42.464523, 42.464523, 42.464523, 42.464523, 42.464523, 42.464523,
       42.464523, 42.464523, 42.464523, 42.464523, 42.464523, 42.464523,
       42.464523, 42.464523, 42.464523, 42.464523, 42.464523, 42.464523,
       42.464523, 42.464523, 42.464523, 42.464523, 42.464523, 42.464523,
       42.464523, 42.464523, 42.464523, 42.464523, 42.464523, 42.464523,
       42.464523, 42.464523, 42.464523, 42.464523, 42.464523, 42.464523,
       42.464523, 42.464523, 42.464523, 42.464523, 42.464523, 42.464523,
       42.464523, 42.464523, 42.464523, 42.464523, 42.464523, 42.464523,
       42.464523, 42.464523, 42.464523, 42.464523, 42.464523, 42.464523,
       42.464523], dtype=float32)

In [33]:
# Reward to go
_gamma=0.99
_use_reward_to_go=True
_use_baseline=False
test=estimate_returns(x2)
test

array([42.464523 , 41.464523 , 40.474525 , 39.494427 , 38.524124 ,
       37.56353  , 36.61254  , 35.67106  , 34.738995 , 33.81625  ,
       32.902733 , 31.99835  , 31.103012 , 30.216627 , 29.339106 ,
       28.47036  , 27.610302 , 26.758844 , 25.915901 , 25.081387 ,
       24.255219 , 23.437311 , 22.627584 , 21.825953 , 21.03234  ,
       20.24666  , 19.46884  , 18.698797 , 17.936453 , 17.181734 ,
       16.434563 , 15.694862 , 14.962559 , 14.237578 , 13.519848 ,
       12.809295 , 12.105846 , 11.409433 , 10.719984 , 10.03743  ,
        9.361701 ,  8.692729 ,  8.030447 ,  7.374788 ,  6.725685 ,
        6.0830736,  5.446888 ,  4.8170643,  4.193539 ,  3.576249 ,
        2.9651318,  2.3601255,  1.7611697,  1.1682032,  0.5811664],
      dtype=float32)

In [34]:
# Baseline
_gamma=0.99
_use_reward_to_go=False
_use_baseline=True
test=estimate_returns(x2)
test

array([-2.1316282e-14, -2.1316282e-14, -2.1316282e-14, -2.1316282e-14,
       -2.1316282e-14, -2.1316282e-14, -2.1316282e-14, -2.1316282e-14,
       -2.1316282e-14, -2.1316282e-14, -2.1316282e-14, -2.1316282e-14,
       -2.1316282e-14, -2.1316282e-14, -2.1316282e-14, -2.1316282e-14,
       -2.1316282e-14, -2.1316282e-14, -2.1316282e-14, -2.1316282e-14,
       -2.1316282e-14, -2.1316282e-14, -2.1316282e-14, -2.1316282e-14,
       -2.1316282e-14, -2.1316282e-14, -2.1316282e-14, -2.1316282e-14,
       -2.1316282e-14, -2.1316282e-14, -2.1316282e-14, -2.1316282e-14,
       -2.1316282e-14, -2.1316282e-14, -2.1316282e-14, -2.1316282e-14,
       -2.1316282e-14, -2.1316282e-14, -2.1316282e-14, -2.1316282e-14,
       -2.1316282e-14, -2.1316282e-14, -2.1316282e-14, -2.1316282e-14,
       -2.1316282e-14, -2.1316282e-14, -2.1316282e-14, -2.1316282e-14,
       -2.1316282e-14, -2.1316282e-14, -2.1316282e-14, -2.1316282e-14,
       -2.1316282e-14, -2.1316282e-14, -2.1316282e-14], dtype=float32)

In [35]:
# Baseline & reward-to-go
_gamma=0.99
_use_reward_to_go=True
_use_baseline=True
test=estimate_returns(x2)
test

array([ 22.791773  ,  21.791773  ,  20.801773  ,  19.821672  ,
        18.851374  ,  17.890778  ,  16.939787  ,  15.998307  ,
        15.066242  ,  14.143497  ,  13.22998   ,  12.325598  ,
        11.43026   ,  10.543875  ,   9.666354  ,   8.797608  ,
         7.93755   ,   7.086092  ,   6.243149  ,   5.408635  ,
         4.5824666 ,   3.7645595 ,   2.9548316 ,   2.153201  ,
         1.3595868 ,   0.5739087 ,  -0.20391269,  -0.97395587,
        -1.7362986 ,  -2.4910178 ,  -3.23819   ,  -3.9778903 ,
        -4.7101936 ,  -5.435174  ,  -6.1529045 ,  -6.8634577 ,
        -7.5669055 ,  -8.263319  ,  -8.952767  ,  -9.635323  ,
       -10.311051  , -10.980023  , -11.642305  , -12.297964  ,
       -12.947067  , -13.589679  , -14.225864  , -14.855688  ,
       -15.479213  , -16.096504  , -16.70762   , -17.312626  ,
       -17.911583  , -18.504549  , -19.091585  ], dtype=float32)

**6. Evaluación del algoritmo**